<a href="https://colab.research.google.com/github/mihaelacoste/PHD-Natural-Disasters/blob/main/GBQ_Queries_Doctorat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Useri care au postat in cel putin 2 dataseturi:


In [ ]:
select t1.ownerusername, count(distinct t1.id) as australia_posts, count(distinct t2.id) as hurricane_posts from `sublime-sunspot-410114.PHD.australian_wildfires` t1
inner join
`sublime-sunspot-410114.PHD.hurricane_irma` t2 on t1.ownerusername = t2.ownerusername
group by 1 order by 2 desc --users which posted on 2 datasets -- 1 user

In [ ]:
select t1.ownerusername, count(distinct t1.id) as hurricane_posts, count(distinct t2.id) as turkey_posts from `sublime-sunspot-410114.PHD.hurricane_irma` t1
inner join
`sublime-sunspot-410114.PHD.turkey_earthquake` t2 on t1.ownerusername = t2.ownerusername
group by 1 order by 2 desc --users which posted on 2 datasets----1 user

Distributie nr postari/user/dataset – top users




In [ ]:
select ownerusername, count(distinct id) from `sublime-sunspot-410114.PHD.turkey_earthquake`
group by 1 order by 2 desc --users with multiple posts in the dataset


Distributie hashtags per dataset



In [ ]:
SELECT
  hashtag,
  COUNT(hashtag) AS hashtag_count
FROM
  `sublime-sunspot-410114.PHD.hurricane_irma`
  left join UNNEST(hashtags) AS hashtag
GROUP BY
  hashtag
ORDER BY
  hashtag_count DESC

In [ ]:
SELECT
  hashtag,
  COUNT(distinct id) AS ids
FROM
  `sublime-sunspot-410114.PHD.australianwildfires`
  left join UNNEST(hashtags) AS hashtag
where hashtag is not null and hashtag <> 'australianwildfires'
  group by hashtag
ORDER BY
  ids DESC
  limit 20

Perechi de hashtag-uri co-ocurente


In [ ]:
SELECT
  hashtag1,
  hashtag2,
  COUNT(*) AS pair_count
FROM (
SELECT
  hashtag1,
  hashtag2
FROM
  `sublime-sunspot-410114.PHD.australian_wildfires`
  left join UNNEST(hashtags) AS hashtag1
  left join UNNEST(hashtags) AS hashtag2
 WHERE
  hashtag1 < hashtag2
)
GROUP BY
  hashtag1, hashtag2
ORDER BY
  pair_count DESC;

Hashtag-uri in toate cele trei seturi de date

In [ ]:
SELECT
  t1.hashtag as common_hashtag,
  COUNT(DISTINCT t1.id) AS count_in_table1,
  COUNT(DISTINCT t2.id) AS count_in_table2,
  COUNT(DISTINCT t3.id) AS count_in_table3
FROM
  (
    SELECT DISTINCT id, hashtag
    FROM `sublime-sunspot-410114.PHD.australianwildfires`, UNNEST(hashtags) AS hashtag
  ) t1
JOIN (
  SELECT DISTINCT id, hashtag
  FROM `sublime-sunspot-410114.PHD.alluvioneemiliaromagna`, UNNEST(hashtags) AS hashtag
) t2 on t1.hashtag = t2.hashtag
JOIN (
  SELECT DISTINCT id, hashtag
  FROM `sublime-sunspot-410114.PHD.earthquake_turkey`, UNNEST(hashtags) AS hashtag
) t3
ON t1.hashtag = t2.hashtag and t2.hashtag = t3.hashtag
GROUP BY t1.hashtag

Nr total, median, avg de hashtags per dataset



In [ ]:
SELECT
  COUNT(distinct hashtag) AS hashtag_total_count,
FROM
  `sublime-sunspot-410114.PHD.turkey_earthquake`
  left join UNNEST(hashtags) AS hashtag

In [ ]:
WITH HashtagCounts AS (
  SELECT
    id,
    COUNT(DISTINCT hashtag) AS num_hashtags
  FROM
    `sublime-sunspot-410114.PHD.turkey_earthquake`,
    UNNEST(hashtags) AS hashtag
  GROUP BY
    id
)
SELECT
  SUM(num_hashtags) / COUNT(*) AS avg_hashtags_overall
FROM
  HashtagCounts

Nr total, median, avg de like-uri per dataset


In [ ]:
SELECT
 sum(likesCount) as total_likes,
 avg(likesCount) as avg_likes,
 APPROX_QUANTILES(likescount, 2)[OFFSET(1)] AS median_likescount,
 max(likescount) as max_likes
FROM
  `sublime-sunspot-410114.PHD.australian_wildfires`

In [ ]:
SELECT
 sum(commentsCount) as total_comments,
 avg(commentsCount) as avg_comments,
 APPROX_QUANTILES(commentsCount, 2)[OFFSET(1)] AS median_comments,
 max(commentsCount) as max_comments
FROM
  `sublime-sunspot-410114.PHD.australian_wildfires`

Location, date

In [ ]:
  select date(timestamp) as date, count(distinct id) as count_posts, sum(likescount) as likes
  from `sublime-sunspot-410114.PHD.hurricane_irma`
  group by 1
  order by 2 desc

In [ ]:
  select locationname, count(distinct id) as count_posts, sum(likescount) as likes
  from `sublime-sunspot-410114.PHD.hurricane_irma`
  group by 1
  order by 2 desc

Lungime caption vs engagement

In [ ]:
SELECT id, length(caption) as caption_length, likescount, commentsCount FROM `sublime-sunspot-410114.PHD.hurricane_irma`
order by likescount desc

select avg(caption_length) as avg_caption_length from (
  select id, length(caption) as caption_length from `sublime-sunspot-410114.PHD.hurricane_irma`
)